# Neo4J: Example of fictious bank clients

The following code templates have been accessed from: https://neo4j.com/docs/python-manual/current/

In [4]:
from neo4j import GraphDatabase
import pandas as pd

## Connect to the database

In [5]:
# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687"

# --- Run locally: ---
# Install the required packages (requirements.txt)
# Start Docker Desktop and create an image, e.g. through Anaconda prompt:
# $ docker run -p7474:7474 -p7687:7687 -d -e NEO4J_AUTH=neo4j/secretgraph neo4j:latest
# To access your graph over Neo4J Desktop: Start Neo4J Desktop --> Graph Apps (left navigation) --> Neo4j Browser --> type in the credentials (neo4j/secretgraph) --> connect --> bolt://localhost:7687
# URI = "neo4j://localhost"

AUTH = ("neo4j", "secretgraph")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

## Writing manually into Neo4J

In [5]:
summary = driver.execute_query(
    "CREATE (:Person {name: $name})",
    name="Anna",
    database_="neo4j",
).summary
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created=summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 1 nodes in 3 ms.


/tmp/ipykernel_786/6459211.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  summary = driver.execute_query(


## Writing manually to the database in Neo4j Browser:

CREATE (:Person {name: 'Peter'})

## Querying

In [17]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

# Loop through results and do something with them
#for record in records:
#    print(record.data())  # obtain record as dict

# Summary information
#print("The query `{query}` returned {records_count} records in {time} ms.".format(
#    query=summary.query, records_count=len(records),
#    time=summary.result_available_after
#))



# alle Alice
records, summary, keys = driver.execute_query(
    "MATCH (p:Person {name:'Alice'}) RETURN p.name AS name, p.age AS age"
)

# nach alter filter
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) WHERE p.age > 40 RETURN p.name AS name, p.age AS age"
)

for record in records:
    print(record.data())  # obtain record as dict


{'name': 'Alice', 'age': 42}
{'name': 'Alice', 'age': 42}


/tmp/ipykernel_786/2964279148.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_786/2964279148.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_786/2964279148.py:24: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


## Update the database

In [10]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    SET p.age = $age
    """, name="Alice", age=42,
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'properties_set': 2}.


/tmp/ipykernel_786/3817737189.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


In [8]:
records, summary, keys = driver.execute_query("""
    MATCH (alice:Person {name: $name})
    MATCH (anna:Person {name: $friend})
    CREATE (alice)-[:KNOWS]->(anna)
    """, name="Alice", friend="Anna",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

/tmp/ipykernel_786/571956250.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


Query counters: {'_contains_updates': True, 'relationships_created': 2}.


## Delete from the database

In [7]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    DETACH DELETE p
    """, name="Alice",
    database_="neo4j",
)
print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'nodes_deleted': 1}.


C:\Users\pell\AppData\Local\Temp\ipykernel_32664\3368292491.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


## Query parameters

In [12]:
driver.execute_query(
    "MERGE (:Person {name: $name})",
    name="Alice", age=42,
    database_="neo4j",
)



/tmp/ipykernel_786/229369716.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  driver.execute_query(


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7cf679dbc710>, keys=[])

In [13]:
parameters = {
    "name": "Alice",
    "age": 42
}
records, summary, keys = driver.execute_query(
    "MERGE (:Person {name: $name})",
    parameters_=parameters,
    database_="neo4j",
)


/tmp/ipykernel_786/3759056323.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


In [106]:
#Testing of Transaction data
records, summary, keys = driver.execute_query(
    "MATCH (p:Client) RETURN p.client_id",
    database_="neo4j",
)
#for record in records:
#    print(record.data())  # obtain record as dict

#take client C1 with the edges and nodes connected
records, summary, keys = driver.execute_query(
    "MATCH (c:Client {client_id: 'C454'})<-[d]-(p:Client) RETURN c.client_id, d.transactions, p.client_id",
    database_="neo4j",
)
#for record in records:
#    print(record.data())


#take client C1 with the edges and nodes connected and exclude the connection type KNOWS
records, summary, keys = driver.execute_query(
    "MATCH (c:Client {client_id: 'C454'})-[d]-(p:Client) WHERE type(d) <> 'KNOWS' RETURN c.client_id, d.transactions, type(d), p.client_id",
    database_="neo4j",
)
#for record in records:
#    print(record.data())


# Finding Path
records, summary, keys = driver.execute_query(
    "MATCH (c:Client {client_id:'C1'})-[s]-(co1:Client)-[d]-(co2:Client) RETURN c.client_id, s.transactions, co1.client_id AS client1, d.transactions, co2.client_id as client2",
    database_="neo4j",
)
#for record in records:
#    print(record.data())


# Finding shortes path
records, summary, keys = driver.execute_query("""
    MATCH p = ((c:Client {client_id:'C1'})--(co:Client {client_id:'C254'})) RETURN count(p) AS pathCount, length(p) AS pathLength
    """,
    database_="neo4j",
)
''' 
if records:
    for record in records:
        path_count = record['pathCount']
        path_length = record['pathLength']
        print(f"Path Count: {path_count}, Path Length: {path_length}")
else:
    print("No paths found.")
'''






/tmp/ipykernel_768/2970241070.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_768/2970241070.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_768/2970241070.py:19: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_768/2970241070.py:28: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(
/tmp/ipykernel_768/2970241070.py:37: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise 

' if records:\n    for record in records:\n        path_count = record[\'pathCount\']\n        path_length = record[\'pathLength\']\n        print(f"Path Count: {path_count}, Path Length: {path_length}")\nelse:\n    print("No paths found.")\n'

In [31]:
records, summary, keys = driver.execute_query("""
    MATCH (C1:Client {client_id: $name})
    MATCH (C2:Client {client_id: $friend})
    CREATE (C1)-[:KNOWS]->(C2)
    """, name="C1", friend="C2",
    database_="neo4j",
)

/tmp/ipykernel_768/329595624.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query("""


In [18]:
# Connection details

# --- Run in Codespaces: ---
URI = "bolt://neo4j:7687" # Replace with your Neo4j bolt URI

# --- Run locally: ---
#URI = "neo4j://localhost"

username = "neo4j"  # Replace with your Neo4j username
password = "secretgraph"  # Replace with your Neo4j password

# Connect to the Neo4j database
driver = GraphDatabase.driver(URI, auth=(username, password))

def create_clients_and_transactions(tx, clients, transactions):
    # Create client nodes
    for client in clients['client_id']:
        tx.run("MERGE (c:Client {client_id: $client_id})", client_id=client)
    
    # Create transaction relationships (edges)
    for index, row in transactions.iterrows():
        tx.run(
            """
            MATCH (c1:Client {client_id: $client1}), (c2:Client {client_id: $client2})
            MERGE (c1)-[r:TRANSACTED_WITH {transactions: $transactions}]->(c2)
            """, 
            client1=row['client1'], 
            client2=row['client2'], 
            transactions=row['transactions']
        )

# Load CSV data into pandas DataFrame

# --- Run in Codespaces: ---
clients_df = pd.read_csv('workspace/example_data/swiss_bank_clients.csv')  # Update with correct path
transactions_df = pd.read_csv('workspace/example_data/swiss_bank_transactions.csv')  # Update with correct path

# --- Run locally: ---
# clients_df = pd.read_csv('example_data/swiss_bank_clients.csv')  # Update with correct path
# transactions_df = pd.read_csv('example_data/swiss_bank_transactions.csv')  # Update with correct path

# Load data into Neo4j
with driver.session() as session:
    session.write_transaction(create_clients_and_transactions, clients_df, transactions_df)

# Close the driver connection
driver.close()

print("Data loaded into Neo4j successfully!")


/tmp/ipykernel_786/1406345837.py:44: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_clients_and_transactions, clients_df, transactions_df)
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.CartesianProductWarning} {category: } {title: This query builds a cartesian product between disconnected patterns.} {description: If a part of a query contains multiple disconnected patterns, this will build a cartesian product between all those parts. This may produce a large amount of data and slow down query processing. While occasionally intended, it may often be possible to reformulate the query that avoids the use of this cross product, perhaps by adding a relationship between the different parts or by using OPTIONAL MATCH (identifier is: (c2))} {position: line: 2, column: 13, offset: 13} for query: '\n            MATCH (c1:Client {client_id: $client1}), (c2:Client {client_id: $client2}

Data loaded into Neo4j successfully!
